In [1]:
import sys
import re
sys.path.append("/uscms/home/sbrightt/nobackup/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMeAnalysis/python_analysis/analysisTools")
import plotTools as ptools
import matplotlib.pyplot as plt
import numpy as np
from coffea import hist
import importlib
import os
from analysisTools import fileAnalyzer, Analyzer

ptools.setDefaultStyle(fontsize=14)

In [2]:
histoLoc = "/uscms_data/d3/sbrightt/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMeAnalysis/python_analysis/outputs/signal_2018_aEM"
matchClass = ptools.loadHistoFromFiles(histoLoc,"ele_match_class")
vtxType = ptools.loadHistoFromFiles(histoLoc,"vtx_matchTypeByEle")
info, masses, cts = ptools.getSampleInfo(matchClass,hname="ele_match_class")

In [3]:
delta_map = {}
for pt in masses:
    m, dm = float(pt.split("-")[0]), float(pt.split("-")[1])
    m1 = m - dm/2
    delta = dm/m1
    if delta not in delta_map.keys():
        delta_map[delta] = {}
    delta_map[delta][pt] = info[pt]

In [4]:
outdir = "plots/summary/GenMatch_effs/"
os.makedirs(outdir,exist_ok=True)
plt.figure()

colors = {"1":"tab:blue","10":"tab:orange","100":"tab:green","1000":"tab:red"}
lines = {0.1:"-",0.4:"--"}

for delta in list(delta_map.keys()):
    m1s = []
    effs = {}
    pts = delta_map[delta]
    for pt in list(pts.keys()):
        m, dm = float(pt.split("-")[0]), float(pt.split("-")[1])
        m1 = m - dm/2
        m1s.append(m1)
        for ct in cts:
            if ct not in effs.keys():
                effs[ct] = []
            samp = pts[pt][ct]
            hist = matchClass.integrate("sample",int_range=samp).integrate("set","RL")
            vals = list(hist.values().values())[0]
            nev = vals.sum()
            eff_uniq = vals[3]/nev
            effs[ct].append(eff_uniq)
    m1s = np.array(m1s)
    sort = np.argsort(m1s)
    m1s = m1s[sort]
    for ct in cts:
        yvals = np.array(effs[ct])[sort]
        plt.plot(m1s,yvals,label=r"$c\tau = {0}$ mm, $\Delta = {1}$".format(ct,delta),marker=".",markersize=10,linestyle=lines[delta],color=colors[ct])
plt.yscale("log")
plt.ylim([None,1])
plt.xlim([5,105])
plt.xticks(np.arange(10,110,step=10))
xmin = plt.gca().get_xlim()[0]
plt.axhline(0.5,color="gray",alpha=0.7,zorder=0,linestyle=":")
plt.axhline(0.2,color="gray",alpha=0.7,zorder=0,linestyle=":")
plt.axhline(0.1,color="gray",alpha=0.7,zorder=0,linestyle=":")
plt.axhline(0.05,color="gray",alpha=0.7,zorder=0,linestyle=":")
plt.axhline(0.01,color="gray",alpha=0.7,zorder=0,linestyle=":")
plt.text(xmin+0.5,0.52,"50%",color="gray",alpha=0.7,fontsize=10,zorder=0)
plt.text(xmin+0.5,0.21,"20%",color="gray",alpha=0.7,fontsize=10,zorder=0)
plt.text(xmin+0.5,0.105,"10%",color="gray",alpha=0.7,fontsize=10,zorder=0)
plt.text(xmin+0.5,0.052,"5%",color="gray",alpha=0.7,fontsize=10,zorder=0)
plt.text(xmin+0.5,0.0105,"1%",color="gray",alpha=0.7,fontsize=10,zorder=0)
plt.legend()
plt.xlabel("$m_1$ [GeV]")
plt.ylabel("Double Gen-Match Efficiency")
plt.savefig(outdir+"uniqMatch_eff_by_m1.pdf")
plt.close()

In [5]:
outdir = "plots/summary/GenMatch_effs/"
os.makedirs(outdir,exist_ok=True)
for delta in list(delta_map.keys()):
    m1s = []
    effs_uniq = {}
    effs_one = {}
    effs_degen = {}
    effs_none = {}
    pts = delta_map[delta]
    for pt in list(pts.keys()):
        m, dm = float(pt.split("-")[0]), float(pt.split("-")[1])
        m1 = m - dm/2
        m1s.append(m1)
        for ct in cts:
            if ct not in effs_uniq.keys():
                effs_uniq[ct] = []
                effs_one[ct] = []
                effs_none[ct] = []
                effs_degen[ct] = []
            samp = pts[pt][ct]
            hist = matchClass.integrate("sample",int_range=samp).integrate("set","RL")
            vals = list(hist.values().values())[0]
            nev = vals.sum()
            eff_none = vals[0]/nev
            eff_one = vals[1]/nev
            eff_degen = vals[2]/nev
            eff_uniq = vals[3]/nev
            effs_none[ct].append(eff_none)
            effs_one[ct].append(eff_one)
            effs_degen[ct].append(eff_degen)
            effs_uniq[ct].append(eff_uniq)
    m1s = np.array(m1s)
    sort = np.argsort(m1s)
    m1s = m1s[sort]
    for ct in cts:
        effs_none[ct] = np.array(effs_none[ct])[sort]
        effs_one[ct] = np.array(effs_one[ct])[sort]
        effs_degen[ct] = np.array(effs_degen[ct])[sort]
        effs_uniq[ct] = np.array(effs_uniq[ct])[sort]
    
    for ct in cts:
        y0 = effs_uniq[ct]
        y1 = y0 + effs_one[ct]
        y2 = y1 + effs_degen[ct]
        y3 = y2 + effs_none[ct]
        plt.figure()
        plt.plot(m1s,y0,color="black",marker=".",markersize=0,linewidth=0,alpha=0.7,label=None)
        plt.plot(m1s,y1,color="black",marker=".",markersize=0,linewidth=0,alpha=0.7,label=None)
        plt.plot(m1s,y2,color="black",marker=".",markersize=0,linewidth=0,alpha=0.7,label=None)
        plt.plot(m1s,y3,color="black",marker=".",markersize=0,linewidth=0,alpha=0.7,label=None)
        plt.xlim([m1s[0],m1s[-1]])
        plt.xticks(m1s)
        plt.ylim([0,1])
        plt.grid(axis='x',color="black",linestyle="--")
        plt.fill_between(m1s,y0,0,color="tab:blue",label="Unique Matches")
        plt.fill_between(m1s,y1,y0,color="tab:green",label="One Match")
        plt.fill_between(m1s,y2,y1,color="tab:orange",label="Degenerate")
        plt.fill_between(m1s,y3,y2,color="tab:red",label="No Matches")
        plt.legend(loc="upper left")
        plt.xlabel("$m_1$ [GeV]")
        plt.ylabel("Fraction")
        plt.title(r"$c\tau = {0}$ mm, $\Delta = {1}$".format(ct,delta),fontsize=20)
        plt.savefig(outdir+"eff_stack_ct-{0}_delta-{1}.pdf".format(ct,delta))
        plt.close()

In [10]:
# making stacks of the vertex type distribution

outdir = "plots/summary/VtxMatch_effs/"
sets = ["RL","RLC"]
os.makedirs(outdir,exist_ok=True)

for st in sets:
    for delta in list(delta_map.keys()):
        m1s = []
        effs_RR = {}
        effs_LR = {}
        effs_LL = {}
        effs_RC = {}
        effs_LC = {}
        effs_CC = {}
        effs_none = {}
        pts = delta_map[delta]
        for pt in list(pts.keys()):
            m, dm = float(pt.split("-")[0]), float(pt.split("-")[1])
            m1 = m - dm/2
            m1s.append(m1)
            for ct in cts:
                if ct not in effs_RR.keys():
                    effs_RR[ct] = []
                    effs_LR[ct] = []
                    effs_LL[ct] = []
                    effs_RC[ct] = []
                    effs_LC[ct] = []
                    effs_CC[ct] = []
                    effs_none[ct] = []
                samp = pts[pt][ct]
                hist = vtxType.integrate("sample",int_range=samp).integrate("set",st)
                vals = list(hist.values().values())[0]
                nev = vals.sum()
                eff_none = vals[0]/nev
                eff_RR = vals[1]/nev
                eff_LR = vals[2]/nev
                eff_LL = vals[3]/nev
                eff_RC = vals[4]/nev
                eff_LC = vals[5]/nev
                eff_CC = vals[6]/nev
                effs_none[ct].append(eff_none)
                effs_RR[ct].append(eff_RR)
                effs_LR[ct].append(eff_LR)
                effs_LL[ct].append(eff_LL)
                effs_RC[ct].append(eff_RC)
                effs_LC[ct].append(eff_LC)
                effs_CC[ct].append(eff_CC)
        m1s = np.array(m1s)
        sort = np.argsort(m1s)
        m1s = m1s[sort]
        for ct in cts:
            effs_none[ct] = np.array(effs_none[ct])[sort]
            effs_RR[ct] = np.array(effs_RR[ct])[sort]
            effs_LR[ct] = np.array(effs_LR[ct])[sort]
            effs_LL[ct] = np.array(effs_LL[ct])[sort]
            effs_RC[ct] = np.array(effs_RC[ct])[sort]
            effs_LC[ct] = np.array(effs_LC[ct])[sort]
            effs_CC[ct] = np.array(effs_CC[ct])[sort]
        for ct in cts:
            y0 = effs_none[ct]
            y1 = y0 + effs_RR[ct]
            y2 = y1 + effs_LR[ct]
            y3 = y2 + effs_LL[ct]
            y4 = y3 + effs_RC[ct]
            y5 = y4 + effs_LC[ct]
            y6 = y5 + effs_CC[ct]
            plt.figure()
            plt.xlim([m1s[0],m1s[-1]])
            plt.xticks(m1s)
            #plt.yscale("log")
            plt.ylim([0,1])
            plt.grid(axis='x',color="black",linestyle="--")
            plt.fill_between(m1s,y0,0,color="tab:blue",label="No Vertex")
            plt.fill_between(m1s,y1,y0,color="tab:green",label="RR")
            plt.fill_between(m1s,y2,y1,color="tab:orange",label="LR")
            plt.fill_between(m1s,y3,y2,color="tab:red",label="LL")
            if st == "RLC":
                plt.fill_between(m1s,y4,y3,color="tab:purple",label="RC")
                plt.fill_between(m1s,y5,y4,color="tab:brown",label="LC")
                plt.fill_between(m1s,y6,y5,color="tab:pink",label="CC")
            plt.legend(loc="upper left")
            plt.xlabel("$m_1$ [GeV]")
            plt.ylabel("Fraction")
            plt.title(r"$c\tau = {0}$ mm, $\Delta = {1}$, Set: {2}".format(ct,delta,st),fontsize=20)
            plt.savefig(outdir+"{0}_vtxMatchType_stack_ct-{1}_delta-{2}.pdf".format(st,ct,delta))
            plt.close()